In [177]:
# !pip install -qU langsmith langchain-core langchain-community langchain-openai langchain-qdrant


In [178]:
# !pip install -qU pymupdf ragas


In [179]:
# pip install -U langchain langchain-openai

In [180]:
import chainlit as cl
import os
import tiktoken
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Qdrant
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from operator import itemgetter
from langchain.schema.output_parser import StrOutputParser
from langchain.schema import Document
import pandas as pd

from dotenv import load_dotenv
import getpass
import os

In [181]:


from uuid import uuid4

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")


LangChain API Key: ········


In [182]:
os.environ["LANGCHAIN_PROJECT"] = "AIE4 - Midterm - {uuid4().hex[0:8]}"



In [183]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")


OpenAI API Key: ········


## Setup RAG System with Basic Chunking

In [184]:
def tiktoken_len(text):
    tokens = tiktoken.encoding_for_model("gpt-4o-mini").encode(text)
    return len(tokens)

In [185]:
# ## Chunking Method Token and Splitting by Paragraph

# def load_and_split_pdfs_by_paragraphs(directory, chunk_size=500, chunk_overlap=25):
#     documents = []
    
#     text_splitter = RecursiveCharacterTextSplitter(
#         chunk_size=chunk_size,
#         chunk_overlap=chunk_overlap,
#         length_function=tiktoken_len,
#         separators=["\n\n", "\n"]
#     )
    
#     for filename in os.listdir(directory):
#         if filename.endswith('.pdf'):
#             file_path = os.path.join(directory, filename)
            
#             loader = PyMuPDFLoader(file_path)
#             pdf_documents = loader.load()
            
#             for page_num, page in enumerate(pdf_documents):
#                 splits = text_splitter.split_text(page.page_content)
#                 for i, split in enumerate(splits):
#                     documents.append(Document(
#                         page_content=split,
#                         metadata={
#                             "filename": filename,
#                             "page_number": page_num + 1,
#                             "chunk_number": i + 1
#                         }
#                     ))
    
#     return documents

In [186]:
#  # Load and split PDFs
# docs = load_and_split_pdfs_by_paragraphs(current_directory)
# len(docs)

In [187]:
def load_and_split_pdfs_by_chunks(directory, chunk_size=750, chunk_overlap=25):
    documents = []
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len
    )
    
    for filename in os.listdir(directory):
        if filename.endswith('.pdf'):
            file_path = os.path.join(directory, filename)
            
            loader = PyMuPDFLoader(file_path)
            pdf_documents = loader.load()
            
            for page_num, page in enumerate(pdf_documents):
                splits = text_splitter.split_text(page.page_content)
                for i, split in enumerate(splits):
                    documents.append(Document(
                        page_content=split,
                        metadata={
                            "filename": filename,
                            "page_number": page_num + 1,
                            "chunk_number": i + 1
                        }
                    ))
    
    return documents

In [188]:
# RAG prompt template
RAG_PROMPT = """
CONTEXT:
{context}

QUERY:
{question}

You are a helpful assistant. Use the available context to answer the question. If you can't answer the question, say you don't know.
"""

prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

In [189]:
# Set the directory containing your PDF files
current_directory = os.getcwd()  # Update this path
current_directory

'/Users/danielgutierrez/Midterm'

In [190]:
 # Load and split PDFs
docs = load_and_split_pdfs_by_chunks(current_directory)
len(docs)

603

In [191]:
 # Initialize embedding model
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

In [192]:
 # Create Qdrant vector store
qdrant_vectorstore = Qdrant.from_documents(
    docs,
    embedding_model,
    location=":memory:",
    collection_name="extending_context_window_llama_3",
)

2024-09-21 10:11:44 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:11:44 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:11:45 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:11:46 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:11:46 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:11:47 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:11:48 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:11:48 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:11:49 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:11:49 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:11:50 - HTTP Request: POST

In [193]:
# Create retriever
qdrant_retriever = qdrant_vectorstore.as_retriever()
  

In [194]:
# Create RAG prompt
rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

# Initialize OpenAI chat model
openai_chat_model = ChatOpenAI(model="gpt-4o-mini")

# Create the RAG chain
rag_chain = (
    {"context": itemgetter("question") | qdrant_retriever, "question": itemgetter("question")}
    | rag_prompt
    | openai_chat_model
    | StrOutputParser()
)

In [195]:
rag_chain.invoke({"question" : "What are the pros and cons of AI being regulated?"})

2024-09-21 10:11:51 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:11:54 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


'Based on the provided context, here are some pros and cons of regulating AI:\n\n### Pros of AI Regulation:\n1. **Safety and Efficacy**: Regulations can help ensure that AI systems are safe and effective before deployment, mitigating risks to both users and organizations (as mentioned in the context regarding innovative solutions like risk assessments and auditing mechanisms).\n2. **Accountability and Transparency**: Regulations can enforce accountability and transparency in AI systems, ensuring they are understandable and traceable, which aligns with the principles outlined in the Blueprint for an AI Bill of Rights.\n3. **Protection of Values**: Regulation can help ensure that AI aligns with national values and ethical standards, making it lawful and purposeful (as per the context).\n4. **Stakeholder Engagement**: The call for meaningful stakeholder engagement in program design suggests that regulation can lead to more inclusive and representative AI systems that consider various inte

## Synthetic Data Generation

In [196]:
from langchain_community.document_loaders import PyMuPDFLoader

#documents = PyMuPDFLoader(file_path="https://s2.q4cdn.com/470004039/files/doc_earnings/2024/q3/filing/_10-Q-Q3-2024-As-Filed.pdf").load()

In [197]:
import os
from langchain_community.document_loaders import PyMuPDFLoader

# Directory where the PDF files are stored
directory_path = os.getcwd()

# Loop over all files in the directory and process PDF files
documents = []
for filename in os.listdir(directory_path):
    if filename.endswith(".pdf"):  # Check if the file is a PDF
        file_path = os.path.join(directory_path, filename)
        # Load the PDF using PyMuPDFLoader
        loader = PyMuPDFLoader(file_path=file_path)
        documents.extend(loader.load())  # Load and append documents to the list

# Now `documents` contains the content from all PDF files in the directory
print(f"Loaded {len(documents)} documents from the directory.")


Loaded 137 documents from the directory.


In [198]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

generator_llm = ChatOpenAI(model="gpt-3.5-turbo")
critic_llm = ChatOpenAI(model="gpt-4o-mini")
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

distributions = {
    simple: 0.5,
    multi_context: 0.4,
    reasoning: 0.1
}

In [199]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

primary_qa_llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | qdrant_retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)

In [200]:
testset = generator.generate_with_langchain_docs(documents, 20, distributions, with_debugging_logs=True)


embedding nodes:   0%|          | 0/284 [00:00<?, ?it/s]

2024-09-21 10:11:56 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:11:56 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:11:56 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:11:56 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:11:56 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:11:56 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:11:56 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:11:56 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:11:56 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:11:56 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['AI Bill of Rights', 'Automated systems', 'American people']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['AI Bill of Rights', 'Automated systems', 'American people']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['AI Bill of Rights', 'Automated systems', 'American people']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['AI Bill of Rights', 'Automated systems', 'American people']}


2024-09-21 10:11:56 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:11:57 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:11:57 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['Lisa Feldman Barrett', 'Microsoft Corporation', 'National Association for the Advancement of Colored People', 'Science and Technology Policy Institute', 'University of Michigan Ann Arbor']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['Lisa Feldman Barrett', 'Microsoft Corporation', 'National Association for the Advancement of Colored People', 'Science and Technology Policy Institute', 'University of Michigan Ann Arbor']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['Lisa Feldman Barrett', 'Microsoft Corporation', 'National Association for the Advancement of Colored People', 'Science and Technology Policy Institute', 'University of Michigan Ann Arbor']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['Lisa Feldman Barrett', 'Microsoft Corporation', 'National Association for the Advancement of Colored People', 'Science and Technology Policy Institute', 'University of Michigan Ann Arbor']}


...

[ragas.testset.evolutions.DEBUG] answer generated: {'answer': 'Automated system creators can provide clear user notifications by ensuring that generally accessible plain language documentation is available, describing the overall system functioning, the role of automation, the responsible entity, and clear explanations of outcomes. These notifications should be kept up-to-date, timely, and accessible to users impacted by the system, with a focus on clarity and quality of information. Explanations should be technically valid, meaningful, and useful, tailored to the level of risk based on the context, and provided in a manner that is understandable to users and operators.', 'verdict': 1}
[ragas.testset.evolutions.DEBUG] answer generated: {'answer': 'Automated system creators can provide clear user notifications by ensuring that generally accessible plain language documentation is available, describing the overall system functioning, the role of automation, the responsible entity, and cle

2024-09-21 10:13:18 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[ragas.testset.filters.DEBUG] evolution filter: {'reason': 'Both questions focus on the role of information integrity within the context of GAI risk management, but the second question introduces an additional aspect of preventing illegal content generation, which expands the breadth of inquiry.', 'verdict': 0}
[ragas.testset.filters.DEBUG] evolution filter: {'reason': 'Both questions focus on the role of information integrity within the context of GAI risk management, but the second question introduces an additional aspect of preventing illegal content generation, which expands the breadth of inquiry.', 'verdict': 0}
[ragas.testset.filters.DEBUG] evolution filter: {'reason': 'Both questions focus on the role of information integrity within the context of GAI risk management, but the second question introduces an additional aspect of preventing illegal content generation, which expands the breadth of inquiry.', 'verdict': 0}
[ragas.testset.filters.DEBUG] evolution filter: {'reason': 'B

2024-09-21 10:13:18 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-21 10:13:20 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[ragas.testset.evolutions.DEBUG] answer generated: {'answer': 'GAI risk management addresses information integrity by obtaining input from stakeholder communities to identify unacceptable use, maintaining an updated hierarchy of identified and expected GAI risks, reevaluating organizational risk tolerances, and devising a plan to halt development or deployment of a GAI system that poses unacceptable negative risk. This process helps prevent illegal content generation by establishing policies and mechanisms to prevent GAI systems from generating illegal content, as well as transparent acceptable use policies that address illegal use or applications of GAI.', 'verdict': 1}
[ragas.testset.evolutions.DEBUG] answer generated: {'answer': 'GAI risk management addresses information integrity by obtaining input from stakeholder communities to identify unacceptable use, maintaining an updated hierarchy of identified and expected GAI risks, reevaluating organizational risk tolerances, and devisin

## Generated Data for Evaluation

In [201]:
testset.to_pandas()


,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,How do healthcare navigators assist individual...,[ \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n...,Healthcare navigators assist individuals in fi...,simple,[{'source': '/Users/danielgutierrez/Midterm/Bl...,True
1,How are AI system performance or assurance cri...,[ \n30 \nMEASURE 2.2: Evaluations involving hu...,AI system performance or assurance criteria ar...,simple,[{'source': '/Users/danielgutierrez/Midterm/NI...,True
2,How can organizations enhance content provenan...,[ \n52 \n• \nMonitoring system capabilities an...,Organizations can enhance content provenance t...,simple,[{'source': '/Users/danielgutierrez/Midterm/NI...,True
3,How can concerns of model collapse be mitigate...,[ \n37 \nMS-2.11-005 \nAssess the proportion o...,Assessing the proportion of synthetic to non-s...,simple,[{'source': '/Users/danielgutierrez/Midterm/NI...,True
4,What is the role of the NIST Generative AI Pub...,[ \n \n \nAbout AI at NIST: The National Insti...,The answer to given question is not present in...,simple,[{'source': '/Users/danielgutierrez/Midterm/NI...,True
5,What processes should be established to verify...,[ \n17 \nGOVERN 1.7: Processes and procedures ...,Processes should be established to verify that...,simple,[{'source': '/Users/danielgutierrez/Midterm/NI...,True
6,What is the purpose of conducting adversarial ...,[ \n39 \nMS-3.3-004 \nProvide input for traini...,The purpose of conducting adversarial testing ...,simple,[{'source': '/Users/danielgutierrez/Midterm/NI...,True
7,"How have advocates, researchers, and governmen...",[SECTION TITLE\n \n \n \n \n \n \nApplying The...,"Advocates, researchers, and government organiz...",simple,[{'source': '/Users/danielgutierrez/Midterm/Bl...,True
8,How can a data breach lead to potential identi...,[ \n \n \n \nDATA PRIVACY \nWHY THIS PRINCIPL...,A data breach can lead to potential identity t...,simple,[{'source': '/Users/danielgutierrez/Midterm/Bl...,True
9,What is the purpose of the Blueprint for an AI...,[ \n \n \n \n \n \n \n \n \n \n \n \n \n \nAbo...,The purpose of the Blueprint for an AI Bill of...,simple,[{'source': '/Users/danielgutierrez/Midterm/Bl...,True


In [202]:
testset_df = testset.to_pandas()

In [203]:
testset_df.to_csv('testset_sdg.csv', index=False)

In [204]:
# from langsmith import Client

# client = Client()

# dataset_name = "AI Regulations Base Chunks v5"

# dataset = client.create_dataset(
#     dataset_name=dataset_name,
#     description="Questions about AI regulations"
# )

In [205]:
# for test in testset.to_pandas().iterrows():
#   client.create_example(
#       inputs={
#           "question": test[1]["question"]
#       },
#       outputs={
#           "answer": test[1]["ground_truth"]
#       },
#       metadata={
#           "context": test[0]
#       },
#       dataset_id=dataset.id
#   )

## LangSmith Evaluation 

In [206]:
eval_llm = ChatOpenAI(model="gpt-4o")

In [207]:
# from langsmith.evaluation import LangChainStringEvaluator, evaluate

# qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

# labeled_helpfulness_evaluator = LangChainStringEvaluator(
#     "labeled_criteria",
#     config={
#         "criteria": {
#             "helpfulness": (
#                 "Is this submission helpful to the user,"
#                 " taking into account the correct reference answer?"
#             )
#         }
#     },
#     prepare_data=lambda run, example: {
#         "prediction": run.outputs["output"],
#         "reference": example.outputs["answer"],
#         "input": example.inputs["question"],
#     }
# )

# dope_or_nope_evaluator = LangChainStringEvaluator(
#     "criteria",
#     config={
#         "criteria": {
#             "dopeness": "Is this submission dope, lit, or cool?",
#         }
#     }
# # )

In [208]:
# import time
# from openai import RateLimitError

# def evaluate_with_retry(rag_chain, dataset_name, retry_limit=5):
#     retry_count = 0
#     while retry_count < retry_limit:
#         try:
#             # Call the evaluate function
#             evaluate(
#                 rag_chain.invoke,
#                 data=dataset_name,
#                 evaluators=[
#                     qa_evaluator,
#                     labeled_helpfulness_evaluator,
#                     dope_or_nope_evaluator
#                 ],
#                 metadata={"revision_id": "default_chain"},
#             )
#             break  # If successful, break out of the loop
#         except RateLimitError as e:
#             # Parse the error to get the required wait time
#             error_message = str(e)
#             if "Rate limit reached" in error_message:
#                 # Extract the wait time from the error message (for example: 1.254s)
#                 wait_time = 1.254  # Default or extract from message if it's dynamic

#                 print(f"Rate limit exceeded. Waiting for {wait_time} seconds before retrying...")
#                 time.sleep(wait_time)
#                 retry_count += 1
#             else:
#                 raise  # If it's not a rate limit error, raise it

#     if retry_count == retry_limit:
#         raise Exception("Retry limit reached. Evaluation failed due to repeated rate limiting.")

# # Call the retry wrapper with the rag_chain and dataset name
# evaluate_with_retry(rag_chain, dataset_name)

## Evaluation of RAG Pipeline with RAGAS

In [209]:
test_questions = testset_df["question"].values.tolist()
test_groundtruths = testset_df["ground_truth"].values.tolist()

In [210]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_augmented_qa_chain.invoke({"question" : question})
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])

2024-09-21 10:13:21 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:13:22 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-21 10:13:22 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:13:26 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-21 10:13:26 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:13:28 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-21 10:13:28 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:13:30 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-21 10:13:30 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:13:33 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-21

In [211]:
#answers

In [212]:
#contexts

In [213]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

In [214]:
response_dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truth'],
    num_rows: 20
})

In [215]:
response_dataset[0]

{'question': 'How do healthcare navigators assist individuals in finding and obtaining healthcare coverage?',
 'answer': 'Healthcare navigators assist individuals by helping them navigate online signup forms to choose and obtain healthcare coverage. They are trained individuals or organizations that guide consumers, small businesses, and their employees in exploring health coverage options through the Marketplace, which is a government website. This includes assisting with completing eligibility and enrollment forms.',
 'contexts': ["HUMAN ALTERNATIVES, \nCONSIDERATION, AND \nFALLBACK \nHOW THESE PRINCIPLES CAN MOVE INTO PRACTICE\nReal-life examples of how these principles can become reality, through laws, policies, and practical \ntechnical and sociotechnical approaches to protecting rights, opportunities, and access. \nHealthcare “navigators” help people find their way through online signup forms to choose \nand obtain healthcare. A Navigator is “an individual or organization that's 

In [216]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

In [217]:
results = evaluate(response_dataset, metrics)

Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

2024-09-21 10:14:18 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-21 10:14:18 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-21 10:14:18 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-21 10:14:18 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:14:19 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:14:19 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-21 10:14:19 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-21 10:14:19 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-21 10:14:19 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-21 10:14:19 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 

## Evaluation Results of Basic Chunking

In [218]:
results

{'faithfulness': 0.8642, 'answer_relevancy': 0.8603, 'context_recall': 0.8315, 'context_precision': 0.8694, 'answer_correctness': 0.6774}

In [219]:
results_df = results.to_pandas()
results_df.head()

,question,contexts,answer,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,How do healthcare navigators assist individual...,"[HUMAN ALTERNATIVES, \nCONSIDERATION, AND \nFA...",Healthcare navigators assist individuals by he...,Healthcare navigators assist individuals in fi...,1.000000,0.943345,1.0,0.750000,0.817212
1,How are AI system performance or assurance cri...,[MEASURE 2.3: AI system performance or assuran...,AI system performance or assurance criteria ar...,AI system performance or assurance criteria ar...,1.000000,0.924966,1.0,1.000000,0.769750
2,How can organizations enhance content provenan...,[origins and modiﬁcations. Further narrowing o...,Organizations can enhance content provenance t...,Organizations can enhance content provenance t...,0.937500,0.972429,1.0,0.916667,0.830485
3,How can concerns of model collapse be mitigate...,[9 \nand reduced content diversity). Overly ho...,Concerns of model collapse can be mitigated by...,Assessing the proportion of synthetic to non-s...,0.444444,0.916980,0.5,1.000000,0.665512
4,What is the role of the NIST Generative AI Pub...,[trustworthy development and use of AI. \nAckn...,The NIST Generative AI Public Working Group (G...,The answer to given question is not present in...,0.666667,0.970761,1.0,0.000000,0.926068


In [220]:
results_df.to_csv('results_basic_chunking.csv', index=False)
